#### Installing MS SQL Driver

In [0]:
print("Starting")

Starting

In [0]:
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17


% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0100 983 100 983 0 0 35107 0 --:--:-- --:--:-- --:--:-- 35107
OK
 % Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0100 79 100 79 0 0 1025 0 --:--:-- --:--:-- --:--:-- 1025
0% [Working] Get:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease [4,010 B]
0% [Connecting to security.ubuntu.com] [1 InRelease 4,010 B/4,010 B 100%] [Conn 0% [Connecting to security.ubuntu.com] [Connecting to repos.azul.com] 0% [Connecting to security.ubuntu.com] [Waiting for headers] Get:2 https://packages.microsoft.com/ubuntu/16.04/prod xenial/main amd64 Packages [306 kB]
0% [Connecting to security.ubuntu.com] [2 Packages 0 B/306 kB 0%] [Waiting for  0% [Connecting to security.ubuntu.com] [Waiting for headers]0% [2 Packages store 0 B] [Connecting to security.ubuntu.com] [Waiting for head0% [Connecting to archive.ubuntu.com] [Connecting to security.ubuntu.com (91.18 Hit:3 https://repos.azul.com/zulu/deb stable InRelease
 0% [Waiting for headers] [Waiting for headers] Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
 Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
 Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
 Hit:7 http://security.ubuntu.com/ubuntu focal-security InRelease
 0% [Working]0% [Working]0% [Working]0% [Working]0% [Working]100% [Working] Fetched 310 kB in 3s (115 kB/s)
Reading package lists... 0%Reading package lists... 0%Reading package lists... 0%Reading package lists... 5%Reading package lists... 5%Reading package lists... 6%Reading package lists... 6%Reading package lists... 56%Reading package lists... 56%Reading package lists... 56%Reading package lists... 56%Reading package lists... 61%Reading package lists... 66%Reading package lists... 66%Reading package lists... 72%Reading package lists... 72%Reading package lists... 77%Reading package lists... 77%Reading package lists... 77%Reading package lists... 77%Reading package lists... 78%Reading package lists... 78%Reading package lists... 78%Reading package lists... 78%Reading package lists... 86%Reading package lists... 86%Reading package lists... 91%Reading package lists... 91%Reading package lists... 96%Reading package lists... 96%Reading package lists... 96%Reading package lists... 96%Reading package lists... 97%Reading package lists... 97%Reading package lists... 99%Reading package lists... 99%Reading package lists... Done
Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
 msodbcsql17
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 744 kB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial/main amd64 msodbcsql17 amd64 17.8.1.1-1 [744 kB]
Fetched 744 kB in 0s (15.5 MB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package msodbcsql17.
(Reading database ... (Reading database ... 5%(Reading database ... 10%(Reading database ... 15%(Reading database ... 20%(Reading database ... 25%(Reading database ... 30%(Reading database ... 35%(Reading database ... 40%(Reading database ... 45%(Reading database ... 50%(Reading database ... 55%(Reading database ... 60%(Reading database ... 65%(Reading database ... 70%(Reading database ... 75%(Reading database ... 80%(Reading database ... 85%(Reading database ... 90%(Reading database ... 95%(Reading database ... 100%(Reading database ... 92257 files and directories currently installed.)
Preparing to unpack .../msodbcsql17_17.8.1.1-1_amd64.deb ...
debc

#### Importing Libraries

In [0]:
import pyodbc 
import pandas as pd
import numpy as np
import pickle
#import wandb
import os
import time
import csv
import codecs
import urllib.request
import urllib.error
import sys

from datetime import date, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import preprocessing


#### DB credentials using Secrets

In [0]:
server = dbutils.secrets.get(scope='myblob', key='host')
username   = dbutils.secrets.get(scope='myblob', key='user')
password   = dbutils.secrets.get(scope='myblob', key='kc_db')
database = dbutils.secrets.get(scope='myblob', key='db_name')



In [0]:
# wb_api_key = dbutils.secrets.get("wandb", "api_key")
# wandb.login(key=wb_api_key)

#### DB connection string

In [0]:
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()


#### DB query to the table "kcResultTable"

In [0]:
# a.	DHA Main Blv (1016)
# b.	LHR MM Alam (1010)
# c.	LHR MT (1007)
# d.	LHR Cantt (1019)
# e.	LHR Wapda (1018)

In [0]:
query = ''' SELECT store_name
, store_code
, shift
, bill_date
, item_name
, item_code
, item_category
, item_subcategory
, item_quantity
, city
from dbo.kcResultTable
WHERE dbo.kcResultTable.item_category = 'BAKERY'
AND city = 'LHE'
AND store_code IN ('1007', '1010', '1016', '1018', '1019')
and Year(bill_date) >= 2016 '''
sql_data = pd.read_sql(query, conn)
sql_data.head()


Out[8]:

,store_name,store_code,shift,bill_date,item_name,item_code,item_category,item_subcategory,item_quantity,city
0,KC028 LHR DHA BULVD,1016,MOR,2022-01-11,Red Velvet Cup Cake,70101,BAKERY,SNACKS (Regular),2.0,LHE
1,KC028 LHR DHA BULVD,1016,MOR,2022-01-11,Snickers Cupcake,70195,BAKERY,SNACKS (Regular),2.0,LHE
2,KC028 LHR DHA BULVD,1016,MOR,2022-01-12,HIMALYAN PINK SALT,KC32010C,BAKERY,CHIPS,2.0,LHE
3,KC028 LHR DHA BULVD,1016,MOR,2022-01-12,Apple Pie small,70027,BAKERY,SNACKS (Regular),2.0,LHE
4,KC028 LHR DHA BULVD,1016,MOR,2022-01-12,Baklava,12025,BAKERY,SNACKS (Small),2.0,LHE


In [0]:
#making copy of orignal sql_query DF
#data = sql_data.copy()

In [0]:
data1 = sql_data.copy()
data1["item_quantity"] = data1["item_quantity"].apply(lambda x: 0 if x < 0 else x)
data1.loc[data1.item_quantity == 0]

Out[10]:

,store_name,store_code,shift,bill_date,item_name,item_code,item_category,item_subcategory,item_quantity,city
339,KC009 LHR MM ALAM,1010,NIG,2022-01-11,Chocolate Fudge Cake,30003,BAKERY,DESSERTS,0.0,LHE
487,KC033 LHR WAPDA,1018,EVE,2022-01-14,Tres Leches Cake Slice,30094,BAKERY,DESSERTS,0.0,LHE
623,KC012 LHR MT,1007,MOR,2022-01-13,Mineral Water Small (TA),90014,BAKERY,DRINKS,0.0,LHE
1317,KC012 LHR MT,1007,MOR,2022-01-19,Toffee,12032,BAKERY,SNACKS (Small),0.0,LHE
1536,KC012 LHR MT,1007,MOR,2022-02-01,Chicken Patties,70003,BAKERY,SNACKS (Regular),0.0,LHE
...,...,...,...,...,...,...,...,...,...,...
4801128,KC033 LHR WAPDA,1018,EVE,2022-06-23,Mango Preserve 45 Grm,51015,BAKERY,PRESERVE,0.0,LHE
4801336,KC033 LHR WAPDA,1018,MOR,2022-06-23,Chocolate Marble Cake,30039,BAKERY,DESSERTS,0.0,LHE
4801404,KC009 LHR MM ALAM,1010,MOR,2022-06-23,Bran Bread,11003,BAKERY,BREADS - RUSKS,0.0,LHE
4801470,KC028 LHR DHA BULVD,1016,EVE,2022-06-23,Fudge Delight Pastry,70092,BAKERY,SNACKS (Regular),0.0,LHE


In [0]:
# query_demand = '''select dd.ItemCode, dd.Description , dh.storecode, dd.Quantity, dd.MFGDATE , dh.VoucherDate , dh.ReferenceDate ,dh.ModifyDate
# from dbo.[debit-header] dh
# join dbo.[debit-detail] dd
# on dh.VoucherNo = dd.voucherno
# WHERE YEAR(ReferenceDate) >=2017 '''

# demand = pd.read_sql(query_demand, conn)
# demand.tail()


In [0]:
# demand_by_store = demand.groupby(['VoucherDate','storecode','ItemCode','Description'])['Quantity'].sum().reset_index()
# demand_by_store.tail()

In [0]:
#read_lhr temp data from db table lahore-temprature
query1 = '''SELECT * FROM dbo."lahore-temperature"'''
lhr_temp = pd.read_sql(query1, conn)
lhr_temp.tail()



Out[13]:

,name,datetime,temp,conditions
2526,Lahore,2022-06-05,36.900002,Clear
2527,Lahore,2022-06-06,37.700001,Clear
2528,Lahore,2022-06-07,38.599998,Clear
2529,Lahore,2022-06-08,37.500000,Partially cloudy
2530,Lahore,2022-06-09,37.500000,Partially cloudy


In [0]:
lhr_temp2 = lhr_temp.copy()

In [0]:
lhr_temp = lhr_temp2
lhr_temp = lhr_temp.drop_duplicates('datetime', keep='last').sort_values('datetime').reset_index(drop=True)

In [0]:
max_temp_date_present = lhr_temp['datetime'].max()

In [0]:
date_today = str(date.today())

In [0]:
def get_temp_data(city, start, end):
    # This is the core of our weather query URL
    BaseURL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'

    ApiKey='W9LF7Z7N3XEMMPY2MWTMFZ9LY'
    #UnitGroup sets the units of the output - us or metric
    UnitGroup='metric'

    #Location for the weather data
    Location=f'{city},PK'

    #Optional start and end dates
    #If nothing is specified, the forecast is retrieved. 
    #If start date only is specified, a single historical or forecast day will be retrieved
    #If both start and and end date are specified, a date range will be retrieved
    StartDate = start
    EndDate   = end

    #JSON or CSV 
    #JSON format supports daily, hourly, current conditions, weather alerts and events in a single JSON package
    #CSV format requires an 'include' parameter below to indicate which table section is required
    ContentType="csv"

    #include sections
    #values include days,hours,current,alerts
    Include="days"


    print('')
    print(' - Requesting weather : ')

    #basic query including location
    ApiQuery=BaseURL + Location

    #append the start and end date if present
    if (len(StartDate)):
        ApiQuery+="/"+StartDate
        if (len(EndDate)):
            ApiQuery+="/"+EndDate

    #Url is completed. Now add query parameters (could be passed as GET or POST)
    ApiQuery+="?"

    #append each parameter as necessary
    if (len(UnitGroup)):
        ApiQuery+="&unitGroup="+UnitGroup

    if (len(ContentType)):
        ApiQuery+="&contentType="+ContentType

    if (len(Include)):
        ApiQuery+="&include="+Include

    ApiQuery+="&key="+ApiKey



    print(' - Running query URL: ', ApiQuery)
    print()

    try: 
        CSVBytes = urllib.request.urlopen(ApiQuery)
    except urllib.error.HTTPError  as e:
        ErrorInfo= e.read().decode() 
        print('Error code: ', e.code, ErrorInfo)
        sys.exit()
    except  urllib.error.URLError as e:
        ErrorInfo= e.read().decode() 
        print('Error code: ', e.code,ErrorInfo)
        sys.exit()


    # Parse the results as CSV
    CSVText = csv.reader(codecs.iterdecode(CSVBytes, 'utf-8'))

    RowIndex = 0

    # The first row contain the headers and the additional rows each contain the weather metrics for a single day
    # To simply our code, we use the knowledge that column 0 contains the location and column 1 contains the date.  The data starts at column 4
    for Row in CSVText:
        if RowIndex == 0:
            FirstRow = Row
            df_temperature = pd.DataFrame(columns=FirstRow)
        else:
            #print('Weather in ', Row[0], ' on ', Row[1])
            ColIndex = 0
            for Col in Row:
                if ColIndex >= 4:
                    pass
                    #print('   ', FirstRow[ColIndex], ' = ', Row[ColIndex])
                ColIndex += 1
        RowIndex += 1
        df_temperature.loc[len(df_temperature)] = Row    
    df_temperature.drop([df_temperature.index[0],df_temperature.index[1]], inplace=True)

    # If there are no CSV rows then something fundamental went wrong
    if RowIndex == 0:
        print('Sorry, but it appears that there was an error connecting to the weather server.')
        print('Please check your network connection and try again..')

    # If there is only one CSV  row then we likely got an error from the server
    if RowIndex == 1:
        print('Sorry, but it appears that there was an error retrieving the weather data.')
        print('Error: ', FirstRow)
  
    return df_temperature

In [0]:

prev_missed_temp = get_temp_data('LahoreCity', max_temp_date_present, date_today )

- Requesting weather : 
 - Running query URL: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/LahoreCity,PK/2022-06-25/2022-07-02?&unitGroup=metric&contentType=csv&include=days&key=W9LF7Z7N3XEMMPY2MWTMFZ9LY

In [0]:
max_temp_date_present

Out[20]: '2022-06-25'

In [0]:
prev_missed_temp

Out[21]:

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipprob,precipcover,preciptype,snow,snowdepth,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
2,"Lahore, Pakistan",2022-06-26,38.9,29,34.6,41.7,30.8,36.9,19.5,42.2,0,0,0,,0,0,24.1,17.7,75.6,999.7,0,4.4,348.6,30.1,10,10,2022-06-26T04:59:43,2022-06-26T19:11:31,0.97,Clear,Clear conditions throughout the day.,clear-day,"OPLA,42071099999,VIAR,41640099999"
3,"Lahore, Pakistan",2022-06-27,39.8,31,35.2,51.2,34.9,43.3,24.6,55,0,0,0,,0,0,15.8,12.8,125.5,1000.4,21.4,4.1,339.5,29.3,10,10,2022-06-27T05:00:02,2022-06-27T19:11:37,0.99,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,"OPLA,42071099999,VIAR,41640099999"
4,"Lahore, Pakistan",2022-06-28,40.8,32,36.3,51.1,39.1,44.3,24.5,52.3,0,0,0,,0,0,15.1,17.7,200.6,996.8,19.5,4,342.7,29.8,10,10,2022-06-28T05:00:22,2022-06-28T19:11:41,1,Clear,Clear conditions throughout the day.,clear-day,"OPLA,42071099999,VIAR,41640099999"
5,"Lahore, Pakistan",2022-06-29,40,33,36.5,48.1,40,44.3,24.3,50.8,0,0,0,,0,0,42.1,36.4,148.7,993.5,23.2,4.1,339.7,29.3,10,10,2022-06-29T05:00:43,2022-06-29T19:11:43,0,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"OPLA,42071099999,VIAR,41640099999"
6,"Lahore, Pakistan",2022-06-30,32.4,27,30,42.2,29.5,36.4,25.1,75.5,0.1,100,4.17,rain,0,0,49.7,40.3,125.4,996,67.4,4,257.5,22.1,10,60,2022-06-30T05:01:06,2022-06-30T19:11:44,0.01,"Rain, Partially cloudy",Partly cloudy throughout the day with afternoo...,rain,"OPLA,42071099999,VIAR,41640099999"
7,"Lahore, Pakistan",2022-07-01,29,25,27,33.6,25,28.7,23,79.1,2.2,100,33.33,rain,0,0,29.9,31.2,115.9,999.5,82.5,5,126.9,10.9,5,30,2022-07-01T05:01:29,2022-07-01T19:11:44,0.03,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"OPLA,VIAR"
8,"Lahore, Pakistan",2022-07-02,38.1,26,32.5,42.3,26,35.4,21.6,56.1,1.5,100,25,rain,0,0,22,12.6,119.8,996.8,66.2,14.8,199.7,17.3,8,10,2022-07-02T05:01:53,2022-07-02T19:11:42,0.05,"Rain, Partially cloudy",Partly cloudy throughout the day with early mo...,rain,"OPLA,VIAR"


In [0]:
prev_missed_temp = prev_missed_temp[['name','datetime','temp','conditions']]

In [0]:
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

for index, row in prev_missed_temp.iterrows():
        cursor.execute('''INSERT INTO dbo."lahore-temperature"  values(?,?,?,?)''', "Lahore", row['datetime'] , row['temp'],row['conditions'])
    
conn.commit()
cursor.close()

In [0]:
# #read_isb temp data from db table islamabad-temprature
# query2 = '''SELECT * FROM dbo."islamabad-temperature"'''
# isb_temp = pd.read_sql(query2, conn)
# isb_temp.tail()



In [0]:
lhr_temp = pd.concat([lhr_temp, prev_missed_temp], ignore_index=True)

In [0]:
#make label encoders for temp data for lahore and islamabad

le_lhr = preprocessing.LabelEncoder()
#le_isb = preprocessing.LabelEncoder()

lhr_temp['conditions'] = le_lhr.fit_transform(lhr_temp['conditions'])
#isb_temp['conditions'] = le_isb.fit_transform(isb_temp['conditions'])

lhr_temp.tail()



Out[26]:

,name,datetime,temp,conditions
2491,"Lahore, Pakistan",2022-06-28,36.3,0
2492,"Lahore, Pakistan",2022-06-29,36.5,2
2493,"Lahore, Pakistan",2022-06-30,30,5
2494,"Lahore, Pakistan",2022-07-01,27,5
2495,"Lahore, Pakistan",2022-07-02,32.5,5


Out[41]:

,name,datetime,temp,conditions
2477,Lahore,2022-06-14,38.900002,0
2478,Lahore,2022-06-15,41.000000,2
2479,Lahore,2022-06-16,39.299999,5
2480,Lahore,2022-06-17,30.000000,5
2481,Lahore,2022-06-18,30.900000,5


In [0]:
#read_holidays data from db table holidays
query3 = '''SELECT * FROM dbo.holidaysData'''
holidays = pd.read_sql(query3, conn)
holidays.tail()



Out[27]:

,date,name,holiday_category
2914,2022-12-27,non national holiday,1.0
2915,2022-12-28,non national holiday,1.0
2916,2022-12-29,non national holiday,1.0
2917,2022-12-30,non national holiday,1.0
2918,2022-12-31,non national holiday,1.0


In [0]:
#convert holidays date column to datetime
holidays.date = pd.to_datetime(holidays.date)
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 3 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 date 2919 non-null datetime64[ns]
 1 name 2919 non-null object 
 2 holiday_category 2919 non-null float64 
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 68.5+ KB

In [0]:
def timeseries_evaluation_metrics_func(y_true, y_pred):
    print(f'MSE is : {mean_squared_error(y_true, y_pred)}')
    print(f'MAE is : {mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE is : {np.sqrt(mean_squared_error(y_true, y_pred))}')

 

In [0]:
from neuralprophet import NeuralProphet

#### temprature scrapping for next 7 days

In [0]:
# lhr_data = get_temp_data('Lahorecity', '2022-03-01', '2022-06-09')

In [0]:
# lhr_data = lhr_data[['name','datetime','temp','conditions']]
# lhr_data['name'] = 'Lahore'

In [0]:
# conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
# cursor = conn.cursor()

# for index, row in lhr_data.iterrows():
#     cursor.execute('''INSERT INTO dbo."lahore-temperature"  values(?,?,?,?)''', "Lahore", row['datetime'], row['temp'], row['conditions'])
    
# conn.commit()
# cursor.close()

#### model definition

In [0]:
def makeModel():
    m = NeuralProphet(
        n_forecasts = 1,
        growth='linear',
        changepoints=None, # list of dates that may include change points (None -> automatic )
        n_changepoints=10,
        changepoints_range=0.2,
        trend_reg=0.5,
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True,
        seasonality_mode="multiplicative",
        #ar_sparsity = 0.5,
        seasonality_reg=0.5,
        n_lags=1,
        num_hidden_layers=7,
        d_hidden=12,
        learning_rate=0.01,
        epochs=130,
        loss_func="MSE",
        normalize="standardize",
        impute_missing=True,
        optimizer = 'AdamW'
      )
    m.add_lagged_regressor("temp")
    m.add_lagged_regressor("conditions")
    #m.add_lagged_regressor("demand_class")
    m.add_lagged_regressor("holiday_category")

    return m



In [0]:
config = { "n_forecasts" : 1,
"growth":'linear',
"changepoints":None,
"n_changepoints":10,
"changepoints_range":0.2,
"trend_reg":0.5,
"yearly_seasonality":True,
"weekly_seasonality":True,
"daily_seasonality":True,
"seasonality_mode":"multiplicative",
"ar_sparsity" : 0.5,
"seasonality_reg":0.5,
"n_lags":1,
"num_hidden_layers":5,
"d_hidden":10,
"learning_rate":0.01,
"epochs":130,
"loss_func":"MSE",
"normalize":"standardize",
"impute_missing":True,
"optimizer" : 'AdamW'
    }

#### for mean imputation w.r.t sku for last N days

In [0]:
def calculate_prev_mean (df_sku_group, df_above_avg):
    N_DAYS_AGO = 30
    #print(df_above_avg[df_above_avg.y == df_above_avg.y.max()])
    for i in range(0, len(df_above_avg)):
        df_single_record = df_above_avg.iloc[i]
        date_from_df = df_single_record.ds
        date_n_days_ago = date_from_df - timedelta(days=N_DAYS_AGO)
        df_temp_average = df_sku_group.loc[(df_sku_group['ds'] >= str(date_n_days_ago)) & (df_sku_group['ds'] < str(date_from_df))]
        average_quantity = round(df_temp_average['y'].mean(), 0)
        #print(f"new avg for {df_single_record.y} ->", average_quantity)
        df_sku_group.loc[df_sku_group.ds == str(date_from_df) , "y"] = average_quantity

 

#### model building processing/training

In [0]:
def model_build (data, list_SKUs, path, temp_data, future_temp_df, holidays_df, city) :
        
        y_zeros = [0.0 for x in range(30)]
        #demand_class = [4.0 for x in range(30) ]
        
        predictions = pd.DataFrame(columns=['pred_date', 'item_code' ,'item_name', 'quantity', 'item_subcategory' ,'city'])
        cityName ='LHE'
        if  'Islamabad' in city:
            cityName = 'ISB'

        temp_data.datetime = pd.to_datetime(temp_data.datetime)


        for sku in list_SKUs:
            print(f"\n\t\t\t==>SKU {sku}\n")

            #print("--> before slicing", data.shape)
            #dataframes slicing
            temp = data[data.item_code == sku] 
            sku_name = temp.item_name.unique()[0]
            sub_category = temp.item_subcategory.unique()[0]
            storeCode = temp['store_code'].unique()[0]
            shift_ = temp['shift'].unique()[0]
            experimentName = f"KC_{storeCode}_{sku}_{shift_}" 
            dir_path = path
            isExist = os.path.exists(dir_path)
            if not isExist: # Create a new directory because it does not exist 
                os.makedirs(dir_path)
                print(f"path created for sku:{sku}")  
            #mlflow.set_experiment(experimentName)

            temp = temp.drop_duplicates(['ds'])
            temp = temp.set_index('ds')
            temp = temp.sort_index()
            temp = temp.reset_index()
            df_average = temp[(temp.ds>='2021-06-01') & (temp.ds < '2021-12-15')]
            average_quantity = round(df_average['y'].mean(), 0)
            if df_average.empty:
                average_quantity = round(temp['y'].mean(), 0)
                if average_quantity > 200:
                    average_quantity = 100

            df_above_avg = temp.loc [temp['y'] > average_quantity]
            calculate_prev_mean(temp, df_above_avg)
            
            global df_newData
            df_newData = pd.concat([df_newData,temp], ignore_index=True)
            
            temp = temp.set_index('ds')
            temp_N_20 = temp['2022-01-25':]
            temp = temp[:'2022-01-25']
            
            
            print("---> after slice and removing duplicates" , temp.shape)
            if len(temp) > 50: 
                filled = temp.asfreq('D') #extending and adding 0 on missing days
                filled = filled.fillna(0 , axis=0)
                temp_df = filled.reset_index()
                #merge holidays data
                new_df = pd.merge(temp_df, holidays_df, left_on='ds',right_on='date')
                #merge temprature data
                temp_df1 = pd.merge(new_df, temp_data, left_on='ds',right_on='datetime')
                temp_df1["temp"] = pd.to_numeric(temp_df1["temp"], downcast="float")
                temp_df1["conditions"] = pd.to_numeric(temp_df1["conditions"], downcast="float")
                temp_df1["holiday_category"] = pd.to_numeric(temp_df1["holiday_category"], downcast="float")
                #temp_df1["demand_class"] = pd.to_numeric(temp_df1["demand_class"], downcast="float")
                
                model = makeModel()

                #train = temp_df1[['ds','y','temp','conditions', 'demand_class' ,'holiday_category']]
                train = temp_df1[['ds','y','temp','conditions' ,'holiday_category']]
                df_train, df_test = model.split_df(train, freq='D', valid_p = 0.05)
                metrics = model.fit(df_train, freq='D', validation_df=df_test)
                future = model.make_future_dataframe(df_train, periods=len(df_test), n_historic_predictions=len(df_train)-len(df_test) )
                forecast = model.predict(future)
                fig3 = model.plot(forecast, xlabel="Date", ylabel="quantity", figsize=(12,7))
                plt.title(sku_name)
                # model pickle save
                with open(dir_path + experimentName + ' model.pkl', "wb") as f:
                    print("\n===>",'model.pkl is saved')
                    pickle.dump(model, f)
                    fig3.savefig(dir_path + experimentName +' InSample.jpg', bbox_inches='tight')       
                    plt.close(fig3)

                frcst_temp = future_temp_df[['datetime','temp', 'conditions']]
                frcst_temp['datetime'] = pd.to_datetime(frcst_temp['datetime'])
                if len(temp_N_20) > 1 :
                    temp_20 = temp_N_20.reset_index()[['ds','y']]
                    frcst_temp = pd.merge(frcst_temp, temp_20,  how='left', left_on='datetime',right_on='ds')
                    frcst_temp.y = frcst_temp.y.fillna(0)
                    print("================>log: Its workings! ")
                else:
                    frcst_temp['y'] = y_zeros[:len(frcst_temp)]
                    print("================>log: Its not workings ")
                
                frcst_temp = pd.merge(frcst_temp, holidays_df, left_on='datetime',right_on='date')
                #frcst_temp['demand_class'] = demand_class[:len(frcst_temp)] 
                #frcst_temp =  frcst_temp[['datetime','y','temp', 'conditions','demand_class' ,'holiday_category']]
                frcst_temp =  frcst_temp[['datetime','y','temp', 'conditions' ,'holiday_category']]
                

                if  'Islamabad' in city:
                    #print(le_isb.classes_)
                    #print(le_isb.get_params())
                    frcst_temp.conditions = le_isb.transform(frcst_temp.conditions)
                else:
                    frcst_temp.conditions = le_lhr.transform(frcst_temp.conditions)
                frcst_temp["temp"] = pd.to_numeric(frcst_temp["temp"], downcast='float')
                frcst_temp["conditions"] = pd.to_numeric(frcst_temp["conditions"], downcast='float')
                frcst_temp["holiday_category"] = pd.to_numeric(frcst_temp["holiday_category"], downcast='float')
                #frcst_temp["demand_class"] = pd.to_numeric(frcst_temp["demand_class"], downcast='float')

                frcst_temp["y"] = pd.to_numeric(frcst_temp["y"], downcast='float')
                frcst_temp = frcst_temp.rename(columns={'datetime': 'ds'})
                print(frcst_temp)
                future = model.make_future_dataframe(frcst_temp, n_historic_predictions=True, periods=10)
                forecast = model.predict(future)
                forecast['item_name'] = sku_name
                forecast['item_code'] = sku 
                forecast['city'] = cityName
                forecast['item_subcategory'] = sub_category
                #print(forecast.head())
                result = forecast[['ds','item_code','item_name','yhat1', 'item_subcategory' ,'city']]
                result = result.rename(columns={'ds': 'pred_date', 'yhat1': 'quantity'})
                predictions= pd.concat([predictions,result.iloc[1:,:]],ignore_index=True)
                                        
                
            else:
                print(f"this sku '{sku_name}' has less records, only {len(temp)}")

        return predictions

#### main model loop

In [0]:
start = date.today() -  timedelta(days=1) 
end = date.today() +  timedelta(days=8)

start, end

Out[38]: (datetime.date(2022, 7, 1), datetime.date(2022, 7, 10))

In [0]:
future_lhr_temp = get_temp_data('LahoreCity', str(start), str(end) )
#future_isb_temp = get_temp_data('Islamabad', str(start), str(end) )
future_lhr_temp = future_lhr_temp.reset_index()
#future_isb_temp = future_isb_temp.reset_index()


- Requesting weather : 
 - Running query URL: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/LahoreCity,PK/2022-07-01/2022-07-10?&unitGroup=metric&contentType=csv&include=days&key=W9LF7Z7N3XEMMPY2MWTMFZ9LY

In [0]:
# loop through the item code, in dataframe to make subsets,
# make a funtion to perform 
  # Sum all the sales w.r.t the dates for looping Sku
  # Call the NP model to forecast the sales for that particular SKU
  # check the MAE/MSE metrics
  # save model 
  # make predictions for next 7 days

directory = '/dbfs/FileStore/models/dated ' + str(date.today()) + '/'

predictions = pd.DataFrame(columns=['pred_date','item_code', 'item_name', 'quantity' , 'item_category', 'item_subcategory', 'shift' , 'store_code', 'store_name', 'city'])



#slicing city
df_city = data1.copy()
df_newData = pd.DataFrame()

#all_projects = wandb.Api().projects(per_page=500)
#logged_skus = list()

#top store
df_store = df_city.groupby('store_code')['item_quantity'].sum().reset_index()
df_store.sort_values('item_quantity', ascending =False, inplace=True, ignore_index = True)
top_store = df_store.store_code.iloc[:5] #setting stores to for predictions
#top_store = ['1005']
#Top skus in top stores for each shift
#wb_api_key = dbutils.secrets.get("wandb", "api_key")
for store in top_store:
    df_temp = df_city.loc[df_city.store_code == store]
    store_name = df_temp.store_name.unique()[0]
    for shifts in ['MOR', 'EVE']:
        df_shift = df_temp.loc[df_temp['shift'] == shifts]
        # get the city name
        city = df_shift.city.unique()[0]
        print("\n===> Shift: ", shifts," in Store: ", store_name, " City: ", city ,"\n")
        df_sku_group = df_shift.groupby(['bill_date','item_code','item_name','item_category','item_subcategory','store_name', 'store_code', 'city','shift'])['item_quantity'].sum().reset_index()
        sku_group = df_shift.groupby(['item_code'])['item_quantity'].sum().reset_index()
        new_sku_df = sku_group.sort_values('item_quantity', ascending = False, ignore_index= True)
         #set numbers of SKUs to get
        top_sku = new_sku_df.item_code.iloc[:100] #setting no# skus 
        #top_sku = ['12003']
        
        if city == "LHE":
            #make a path
            path = directory + 'Lahore/store_' + store + "/"
            #make a df
            df = df_sku_group.rename(columns={'bill_date': 'ds', 'item_quantity': 'y' })
            #call model build functions result = model_build(temp_df, path, lhr_temp,'LahoreCity')
            result = model_build(df, top_sku , path, lhr_temp, future_lhr_temp , holidays ,'LahoreCity')
        else:
            #make a path
            path = directory + 'Islamabad/store_' + store + "/"
            #make a df
            df = df_sku_group.rename(columns={'bill_date': 'ds', 'item_quantity': 'y' })
            #call model build functions result = model_build(temp_df, path, lhr_temp,'LahoreCity')
            result = model_build(df, top_sku, path, isb_temp,future_isb_temp, holidays, 'Islamabad', wb_api_key)

        result['shift'] =  shifts
        result['store_code'] = store
        result['store_name'] = store_name
        result['item_category'] = 'BAKERY'
        result = result[['pred_date','item_code', 'item_name', 'quantity', 'item_category', 'item_subcategory', 'shift' , 'store_code', 'store_name', 'city']]
        predictions = pd.concat([predictions,result],ignore_index=True)


===> Shift: MOR in Store: KC033 LHR WAPDA City: LHE 


 ==>SKU 70003

path created for sku:70003
---> after slice and removing duplicates (1944, 9)
INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO - (NP.config.__post_init__) - Note: Fourier-based seasonality regularization is experimental.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.952% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.949% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
 0%| | 0/130 [00:00<?, ?it/s]Epoch[1/130]: 0%| | 0/130 [00:00<?, ?it/s]Epoch[1/130]: 0%| | 0/130 [00:00<?, ?it/s, MSELoss=6.75, MAE=15.9, RMSE=19.8, RegLoss=0, MAE_val=19.5, MSELoss_val=9.07, RMSE_val=23.1]Epoch[1/130]: 1%| | 1/130 [00:00<01:22, 1.56it/s, MSELoss=6.75, MAE=15.9, RMSE=19.8, RegLoss=0, MAE_val=19.5, MSELoss_val=9.07, RMSE_val=23.1]Epoch[2/130]: 1%| | 1/130 [00:01<01:22, 1.56it/s, MSELoss=6.75, MAE=15.9, RMSE=19.8, RegLoss=0, MAE_val=19.5, MSELoss_val=9.07, RMSE_val=23.1]Epoch[2/130]: 1%| | 1/130 [00:01<01:22, 1.56it/s, MSELoss=5.63, MAE=14.5, RMSE=18, RegLoss=0, MAE_val=17.8, MSELoss_val=7.56, RMSE_val=21.1] Epoch[2/130]: 2%|▏ | 2/130 [00:01<01:21, 1.57it/s, MSELoss=5.63, MAE=14.5, RMSE=18, RegLoss=0, MAE_val=17.8, MSELoss_val=7.56, RMSE_val=21.1]Epoch[3/130]: 2%|▏ | 2/130 [00:01<01:21, 1.57it/s, MSELoss=5.63, MAE=14.5, RMSE=18, RegLoss=0, MAE_val=17.8, MSELoss_val=7.56, RMSE_val=21.1]Epoch[3/130]: 2%|▏ | 2/130 [00:01<01:21, 1.57it/s, MSELoss=4.97, MAE=13.6, RMSE=17, RegLoss=0, MAE_val=16.6, MSELoss_val=6.51, RMSE_val=19.6]Epoch[3/130]: 2%|▏ | 3/130 [00:01<01:21, 1.56it/s, MSELoss=4.97, MAE=13.6, RMSE=17, RegLoss=0, MAE_val=16.6, MSELoss_val=6.51, RMSE_val=19.6]Epoch[4/130]: 2%|▏ | 3/130 [00:02<01:21, 1.56it/s, MSELoss=4.97, MAE=13.6, RMSE=17, RegLoss=0, MAE_val=16.6, MSELoss_val=6.51, RMSE_val=19.6]Epoch[4/130]: 2%|▏ | 3/130 [00:02<01:21, 1.56it/s, MSELoss=4.45, MAE=12.8, RMSE=16, RegLoss=0, MAE_val=15.4, MSELoss_val=5.52, RMSE_val=18] Epoch[4/130]: 3%|▎ | 4/130 [00:02<01:20, 1.56it/s, MSELoss=4.45, MAE=12.8, RMSE=16, RegLoss=0, MAE_val=15.4, MSELoss_val=5.52, RMSE_val=18]Epoch[5/130]: 3%|▎ | 4/130 [00:03<01:20, 1.56it/s, MSELoss=4.45, MAE=12.8, RMSE=16, RegLoss=0, MAE_val=15.4, MSELoss_val=5.52, RMSE_val=18]Epoch[5/130]: 3%|▎ | 4/130 [00:03<01:20, 1.56it/s, MSELoss=3.97, MAE=12.1, RMSE=15.1, RegLoss=0, MAE_val=14.3, MSELoss_val=4.7, RMSE_val=16.6]Epoch[5/130]: 4%|▍ | 5/130 [00:03<01:20, 1.55it/s, MSELoss=3.97, MAE=12.1, RMSE=15.1, RegLoss=0, MAE_val=14.3, MSELoss_val=4.7, RMSE_val=16.6]Epoch[6/130]: 4%|▍ | 5/130 [00:03<01:20, 1.55it/s, MSELoss=3.97, MAE=12.1, RMSE=15.1, RegLoss=0, MAE_val=14.3, MSELoss_val=4.7, RMSE_val=16.6]Epoch[6/130]: 4%|▍ | 5/130 [00:03<01:20, 1.55it/s, MSELoss=3.5, MAE=11.4, RMSE=14.3, RegLoss=0, MAE_val=13.2, MSELoss_val=3.95, RMSE_val=15.2]Epoch[6/130]: 5%|▍ | 6/130 [00:03<01:19, 1.55it/s, MSELoss=3.5, MAE=11.4, RMSE=14.3, RegLoss=0, MAE_val=13.2, MSELoss_val=3.95, RMSE_val=15.2]Epoch[7/130]: 5%|▍ | 6/130 [00:04<01:19, 1.55it/s, MSELoss=3.5, MAE=11.4, RMSE=14.3, RegLoss=0, MAE_val=13.2, MSELoss_val=3.95, RMSE_val=15.2]Epoch[7/130]: 5%|▍ | 6/130 [00:04<01:19, 1.55it/s, MSELoss=3.05, MAE=10.7, RMSE=13.3, RegLoss=0, MAE_val=11.6, MSELoss_val=3.09, RMSE_val=13.5]Epoch[7/130]: 5%|▌ | 7/130 [00:04<01:18, 1.57it/s, MSELoss=3.05, MAE=10.7, RMSE=13.3, RegLoss=0, MAE_val=11.6, MSELoss_val=3.09, RMSE_val=13.5]Epoch[8/130]: 5%|▌ | 7/130 [00:05<01:18, 1.57it/s, MSELoss=3.05, MAE=10.7, RMSE=13.3, RegLoss=0, MAE_val=11.6, MSELoss_val=3.09, RMSE_val=13.5]Epoch[8/130]: 5%|▌ | 7/130 [00:05<01:18, 1.57it/s, M

In [0]:
print("training completed!")

training completed!

In [0]:
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

res = cursor.execute("SELECT max(pred_date) FROM dbo.predictions")


In [0]:
max_date = res.fetchone()

In [0]:
max_date[0]


Out[94]: datetime.datetime(2022, 6, 19, 0, 0)

In [0]:
from datetime import datetime
from dateutil import parser


Out[101]: False

####dumping next 7 days predictions into db

In [0]:
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()
ignored = []
for index, row in predictions.iterrows():
    if max_date[0] < row.pred_date:
        cursor.execute("INSERT INTO dbo.predictions  values(?,?,?,?,?,?,?,?,?,?)", row.pred_date, row.item_code , row.item_name, max(0,row.quantity), row.item_category, row.item_subcategory, row['shift'], row.store_code, row.store_name, row.city)
    
conn.commit()
cursor.close()

In [0]:
max_date[0].strftime("%Y-%m-%d")

Out[121]: '2022-06-19'

In [0]:
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

query_predictions = f'''
INSERT INTO itemDemandPredictions (pred_date, item_code, item_name, quantity, item_category, item_subcategory, shift, store_code, store_name, city)
SELECT pred_date, item_code, item_name, quantity, item_category, item_subcategory, shift, store_code, store_name, city FROM predictions
WHERE pred_date > '{max_date[0].strftime("%Y-%m-%d")}'
'''
cursor.execute(query_predictions)
conn.commit()
cursor.close()

In [0]:
predictions.to_csv(directory+'predictions.csv')

In [0]:
print("done!!!")

done!!!